In [3]:
import pandas as pd
import numpy as np
import csv

FS_ALGO = ['dssa', 'f_classif', 'Genetic', 'MRMR', 'New_dssa', 'ReliefF', 'SVM']

def output_data(name):
    df = pd.read_csv(name, header=0)

    df_name = name.split("/")[1].split(".")[0].replace("_results", "")
    CLASSIFIERS = df["Learning algorithm"].unique()
    Ks = df["Number of features selected (K)"].unique()

    for cls in CLASSIFIERS:
        for k in Ks:
            res = []
            scores = df.loc[(df['index'] == 'AUC') & (df['Learning algorithm'] == cls) & (df['Number of features selected (K)'] == k)].groupby("Filtering Algorithm").mean()["value"]
            row = np.array([df_name, cls, k])
            row = np.concatenate((row, scores.values), axis=0)

            scoresdf = pd.DataFrame(scores.values)
            ranks = scoresdf[0].rank(ascending=False)

            row = np.concatenate((row, ranks), axis=0)

            with open("friedman.csv", "a", newline="") as fn:
                write = csv.writer(fn)
                write.writerow(row)

In [28]:
import os

path = 'results/'
all_files = []
for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)):
        all_files.append(os.path.join(path, file))

for name in all_files:
    output_data(name)

In [26]:
def friedman():
    df = pd.read_csv("friedman.csv", header=None)
    ranks = df.iloc[:, -7:]
    average_rank = ranks.mean(axis=0)
    print(average_rank)
    N = df.shape[0]
    L = 7
    sigma = sum([value ** 2 for value in average_rank])

    chi = ((12*N)/(L * (L+1)))*(sigma - (L*((L+1)**2))/4)
    FF = ((N-1)*chi)/(N * (L-1) - chi)

    return N, L, sigma, chi, FF